In [ ]:
import os
import re
import ast
import json
import time

import numpy as np
import pandas as pd
import ollama
from datetime import datetime, timedelta

from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

from langchain_core.tools import tool

from langchain_openai import ChatOpenAI

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document


In [ ]:
@tool
def get_storage_locations_in_plant(plant: str) -> str: # fixed function
    """
    Requests storage locations from the system.
    Args:
        plant (str): The plant to get storage locations for.
    Returns:
        str: A list of storage location ids and their descriptions.
    """
    global STORAGE_LOCATIONS_IN_PLANT
    return STORAGE_LOCATIONS_IN_PLANT[plant]

@tool
def get_pollex_storage_locations() -> str: # fixed function
    """
    Requests pollex storage locations.
    Returns:
        str: A list of storage location ids and their descriptions.
    """
    global POLLEX_STORAGE_LOCATIONS
    return POLLEX_STORAGE_LOCATIONS

@tool
def get_materials_in_storage_location(storage_location: str) -> str: # fixed function
    """
    Requests materials from the system.
    Args:
        storage_location (str): The storage location to get materials from.
    Returns:
        str: A list of materials in the storage location.
    """
    global MATERIALS_IN_STORAGE_LOCATION
    print(MATERIALS_IN_STORAGE_LOCATION)
    return MATERIALS_IN_STORAGE_LOCATION[storage_location]

@tool
def get_storage_location(storage_location_id: str, plant: str) -> str: # fixed function
    """
    Requests information about a storage location.
    Args:
        storage_location_id (str): The id of the storage location to get information about.
        plant (str): The plant to get the storage location from.
    Returns:
        str: The information about the storage location.
    """
    global STORAGE_LOCATION_INFO
    return STORAGE_LOCATION_INFO[plant][storage_location_id]

@tool
def create_storage_location(storage_location_id: str, description: str, plant: str) -> str: # fixed function
    """
    Creates a new storage location with the given description.
    Args:
        storage_location_id (str): The id of the storage location.
        description (str): The description of the storage location.
        plant (str): The plant to create the storage location in.
    Returns:
        str: A message indicating that the storage location was created.
    """
    return 'Storage location with ID {} with description "{}" was created'.format(storage_location_id, description)

@tool
def send_email(email: str, content: str) -> str: # fixed function
    """
    Sends an email to the given email address with the given content.
    Args:
        email (str): The email address to send the email to.
        content (str): The content of the email.
    Returns:
        str: A message indicating that the email was sent.
    """
    return 'Email with content "{}" was send to {}'.format(content, email)

@tool
def transaction_SA38(program: str, program_parameters_dict: dict) -> str: # fixed function
    """
    Executes a transaction SA38 with the passed program and program parameters dictionary.
    Args:
        program (str): The program to execute.
        program_parameters_dict (dict): The parameters dictionary to pass to the program
    Returns:
        str: A message indicating of the transaction was successful.
    """
    return "Transaction {} was successful with parameters: {}".format(program, program_parameters_dict)

@tool
def transaction_MMSC(material_id: str, storage_location_id: str, plant: str) -> str: # fixed function
    """
    Executes a transaction MMSC with the given material id, storage location id and plant.
    Args:
        material_id (str): The id of the material to move.
        storage_location_id (str): The id of the storage location to move the material to.
        plant (str): The plant to move the material in.
    Returns:
        str: A message indicating of the transaction was successful.
    """
    return "Material {} was moved to storage location {} in plant {}".format(material_id, storage_location_id, plant)

@tool
def get_idoc_field(idoc_id: str, segment: str, key: str) -> str: # fixed function
    """
    Get the value of an IDoc field.
    Args:
        idoc_id (str): The id of the IDoc to get the field of.
        segment (str): The segment of the IDoc to get the field of.
        key (str): The key of the IDoc field to get the value of.
    Returns:
        str: The value of the IDoc field.
    """
    global IDOC_VALUES
    return IDOC_VALUES[idoc_id][segment][key]

@tool
def set_idoc_field(idoc_id: str, segment: str, key: str, value: str) -> str: # fixed function
    """
    Sets the value of an IDoc field.
    Args:
        idoc_id (str): The id of the IDoc to set the field of.
        segment (str): The segment of the IDoc to set the field of.
        key (str): The key of the IDoc field to get the value of.
        value (str): The value to set the IDoc field to.
    Returns:
        str: A message indicating if the IDoc field was set or not.
    """
    return "IDoc field {} of segment {} of IDoc {} was set to {}\n".format(key, segment, idoc_id, value)

@tool
def reinitialize_idoc(idoc_id: str) -> str: # fixed function
    """
    Reinitializes an IDoc.
    Args:
        idoc_id (str): The id of the IDoc to reinitialize.
    Returns:
        str: A message indicating if the IDoc was reinitialized or not.
    """
    return "IDoc {} was successfully reinitialized".format(idoc_id)

@tool
def get_current_date() -> str: # fixed function
    """
    Requests the current date.
    Returns:
        str: The current date.
    """
    global CURRENT_DATE
    return CURRENT_DATE

@tool
def rfc(function: str, rfc_parameters: dict) -> str: # function for complex API calls
    """
    Executes an RFC function with the given function and parameters.
    Args:
        function (str): The function to execute.
        rfc_parameters (dict): The parameters to pass to the function.
    Returns:
        str: A message indicating of the RFC function was successful.
    """
    global RFC_READ_TABLE
    FILTER_KEY = [x for x in list(rfc_parameters.keys()) if x != "TABLE_NAME"][0]
    return RFC_READ_TABLE[function][rfc_parameters["TABLE_NAME"]][FILTER_KEY][rfc_parameters[FILTER_KEY]]

@tool
def get_field_by_rpa(transaction: str, rpa_parameters: dict, field: str) -> str: # implemented RPA API
    """
    Retrieves field from an SAP transaction using RPA.
    Args:
        transaction (str): The transaction to get the entry from.
        rpa_parameters (dict): The parameters to pass to the transaction.
        field (str): The field to retrieve from the transaction.
    Returns:
        str: The entry from the SAP field.
    """
    global RPA_FIELD_VALUE
    FILTER_KEY = [x for x in list(rpa_parameters.keys())][0]
    return RPA_FIELD_VALUE[transaction][FILTER_KEY][rpa_parameters[FILTER_KEY]][field]

@tool
def retrieve_most_similar_rule(error_message_description: str) -> str: # most important function
    """
    Retrieves the most similar rule from the rule base for the given error message description.
    Args:
        error_message_description (str): Error message description to retrieve the most similar rule for.
    Returns:
        str: The most similar rule from the rule base.
    """
    global RETRIEVER
    result = RETRIEVER.invoke(error_message_description)[0]
    result = "Error Message Template:\n\n" + result.page_content + "\n\nRule:\n\n" + result.metadata.get("rule")
    return result

@tool
def generate_rule(user_request: str) -> str: # most important function
    """
    Generates a rule based on the user request.
    Args:
        user_request (str): The user request to generate the rule from.
    Returns:
        str: The generated rule.
    """
    llm = ChatOpenAI(
            model="gpt-4o",
            temperature=0,
            seed=42
    )
    response = llm.invoke(f"""You are an assistant that converts user requests into process steps an LLM agent will automatically execute to complete the process. 

    Convert the user_request into unambiguous and pragmatic step that include important step information, decision rules.

    - The steps need to be designed as clear, unambiguous instructions for an LLM agent.
    - Use if-else statements to mark decision points.
    - Nest every step of an if path within the if statement and every else path step within an else statement.
    - Hold the formatting of the steps as simple as possible.

    Convert the following user request into steps according to the instructions above:

    {user_request}""")
    process_rule = response.content
    token_usage = response.response_metadata.get("token_usage", {})
    input_tokens = token_usage.get("prompt_tokens", 0)
    cached_tokens = token_usage.get("prompt_tokens_details", {}).get("cached_tokens", 0)
    output_tokens = token_usage.get("completion_tokens", 0)
    
    global input_tokens_sum, cached_tokens_sum, output_tokens_sum, price
    input_tokens_sum += input_tokens
    cached_tokens_sum += cached_tokens
    output_tokens_sum += output_tokens
    price += (input_tokens - cached_tokens) * 250 / 1000000 + cached_tokens * 125 / 1000000 + output_tokens * 1000 / 1000000
    
    return process_rule


tools = [
    get_storage_locations_in_plant, 
    get_pollex_storage_locations, 
    get_materials_in_storage_location, 
    get_storage_location, 
    create_storage_location, 
    send_email, 
    transaction_SA38, 
    transaction_MMSC, 
    get_idoc_field, 
    get_current_date, 
    set_idoc_field, 
    reinitialize_idoc, 
    rfc, 
    get_field_by_rpa, 
    retrieve_most_similar_rule, 
    generate_rule
]

def create_react_agent(config, sys_msg, tools):

    llm = ChatOpenAI(
        **config
    )

    llm_with_tools = llm.bind_tools(tools)

    def assistant(state: MessagesState):
        return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

    builder = StateGraph(MessagesState)
    builder.add_node("assistant", assistant)
    builder.add_node("tools", ToolNode(tools))


    builder.add_edge(START, "assistant")
    builder.add_edge("tools", "assistant")
    builder.add_conditional_edges("assistant", tools_condition)

    react_graph = builder.compile()

    return react_graph

In [ ]:
SYSTEM_PROMPT = """**Role:**  
You are SAPY an SAP agent responsible for handling user requests and resolving IDOC error messages.
You are only working in the plant 1540. 
Every user request or message you receive is related to this plant 1540.
You will receive the inputs containing either containing:
- error messages in the format below:
    Error Message:
    Idoc ID: <IDOC ID>
    Pollex ID: <Pollex ID>
    Description: <description>

- email responses in the format below:
    <email response content> # this is usually a response to a request you made, stating that the request was completed

    From: >Sender Name> <sender email>
    Date: <date>
    To: LLM Agent <llm.agent@company.com>
    Subject: <subject>

    <original email content>

- user requests with processes to complete:
    <user request content> # this is a request you need to complete (e.g. 'There was an issue with the IDoc 1234. Retrieve...' or 'Please compare the pollex storage locations with the storage locations in plant 1540.')

---

**Task:**
1. Analyze the input.
  - If it is an user request, call the `generate_rule`-tool and follow the generated rule to resolve the request.
  - If it is an email answer to your request stating that a request of you was completed, re-initialize the IDoc.
  - If it is an error message, retrieve the most similar rule to the error message description.
2. Compare the error message template and the rule. Reason step-by-step if this rule can really help you to resolve the original error provided in the input.
  - If the rule seems helpful, follow all steps of this rule to resolve the error efficiently using the tools provided.
    - Reason step-by-step how you can follow the rules and what tools you need to use to follow the rules. 
    - Include the the information from the error message or user request in your reasoning.
    - If a tool error occurs, reason step-by-step how you can solve the tool error and include the following steps:
      - Is the information already provided in the received input or error message?
      - Did you forget any parameters that are necessary for the tool? 
      - Are the parameters formatted correctly?
      - Can you use another tool to complete the step?
      - If the tool error occurs more than 3 times, send an email to error.handler1@company.com with the error message including the Idoc ID, Pollex ID, and Description and a request for help.
  - If you are not sure how to solve the error using the most similar rule or there is no similar rule found, send an email to including the Idoc ID, Pollex ID, and Description and a request for help to error.handler1@company.com

---"""

In [ ]:
def generate_actual_should(x, messages):
    actual_tool_calls = []
    for m in messages["messages"]:
        if type(m) == AIMessage:
            tools = m.additional_kwargs.get("tool_calls", [])
            for tool in tools:
                actual_tool_calls.append([tool.get("function", {}).get("name", ""), ast.literal_eval(tool.get("function", {}).get("arguments", {}))])
    
    mockups = json.loads(x["mockups"])

    should_tool_calls = []
    should_tool_calls.append(["retrieve_most_similar_rule", {"error_message_description": x["error_message"]}])
    if x["process"] == "1.0":
        matches = re.findall(r"Storage location ([0-9]+) is not allowed \(Check your entry\)", x["error_message"])
        if len(matches) == 1:
            storage_location_id = matches[0]
            if int(storage_location_id) > 199:
                should_tool_calls.append(["transaction_SA38", {"program": "RC1_IDOC_SET_STATUS", "program_parameters_dict": {"IDOC_ID": x["idoc_id"], "STATUS": "LKD"}}])
                should_tool_calls.append(["send_email", {"email": "employee3@serviceprovider.com", "content": x["pollex_id"]}])
            else:
                should_tool_calls.append(["get_pollex_storage_locations", {}])
                should_tool_calls.append(["create_storage_location", {"storage_location_id": storage_location_id, "description": f"Location {str(storage_location_id).zfill(4)}", "plant": "1540"}])
                should_tool_calls.append(["reinitialize_idoc", {"idoc_id": x["idoc_id"]}])

    if x["process"] == "2.0":
        matches = re.findall(r"Material ([0-9 ]+) is not available", x["error_message"])
        if len(matches) == 1:
            complete_material_id = matches[0].split(" ")
            material_id = complete_material_id[0].strip()
            plant_id = complete_material_id[1].strip()
            storage_location_id = complete_material_id[2].strip()

            if storage_location_id != "0001":
                should_tool_calls.append(["transaction_MMSC", {"material_id": material_id, "storage_location_id": storage_location_id, "plant": plant_id}])
                should_tool_calls.append(["reinitialize_idoc", {"idoc_id": x["idoc_id"]}])
            else:
                should_tool_calls.append(["transaction_SA38", {"program": "RC1_IDOC_SET_STATUS", "program_parameters_dict": {"IDOC_ID": x["idoc_id"], "STATUS": "LKD"}}])
                should_tool_calls.append(["send_email", {"email": "employee3@serviceprovider.com", "content": x["pollex_id"]}])

    if x["process"] == "3.0":
        matches = re.findall(r"G/L account ([0-9 ]+) is locked for posting in company code ([0-9 ]+)", x["error_message"])
        if len(matches) == 1:
            should_tool_calls.append(["get_current_date", {}])
            should_tool_calls.append(["get_idoc_field", {"idoc_id": x["idoc_id"], "segment": "EDI_DC40", "key": "CREDAT"}])
            if (pd.to_datetime(mockups["CURRENT_DATE"].split(" ")[0]) - timedelta(days=1)).date() == pd.to_datetime(mockups["IDOC_VALUES"][x["idoc_id"]]["EDI_DC40"]["CREDAT"].split(" ")[0]).date():
                should_tool_calls.append(["send_email", {"email": "error.handler2@company.com", "content": x["idoc_id"]}])
            else:
                should_tool_calls.append(["send_email", {"email": "error.handler1@company.com", "content": x["idoc_id"]}])
    
    if x["process"] == "4.0":
        matches = re.findall(r"G/L account ([0-9 ]+) is not provided for in company code ([0-9 ]+)", x["error_message"])
        if len(matches) == 1:
            gl_account_id = matches[0][0]
            company_code = matches[0][1]
            if len(mockups["RFC_READ_TABLE"]["RFC_READ_TABLE"]["GLACCOUNT_SCREEN_KEY"]["SAKNR"].get(gl_account_id, "").split("\n")) == 3:
                should_tool_calls.append(["rfc", {"function": "RFC_READ_TABLE", "rfc_parameters": {"TABLE_NAME": "GLACCOUNT_SCREEN_KEY", "SAKNR": gl_account_id}}])
                should_tool_calls.append(["send_email", {"email": "error.handler1@company.com", "content": x["idoc_id"]}])
            else:    
                should_tool_calls.append(["rfc", {"function": "RFC_READ_TABLE", "rfc_parameters": {"TABLE_NAME": "GLACCOUNT_SCREEN_KEY", "SAKNR": gl_account_id}}])
                should_tool_calls.append(["set_idoc_field", {"idoc_id": x["idoc_id"], "segment": "E1EDKA1", "key": "NAME1", "value": mockups["RFC_READ_TABLE"]["RFC_READ_TABLE"]["GLACCOUNT_SCREEN_KEY"]["SAKNR"][gl_account_id].split(",")[-1].strip()}])
                should_tool_calls.append(["set_idoc_field", {"idoc_id": x["idoc_id"], "segment": "E1EDKA1", "key": "PARTN", "value": mockups["RFC_READ_TABLE"]["RFC_READ_TABLE"]["GLACCOUNT_SCREEN_KEY"]["SAKNR"][gl_account_id].split(",")[-1].strip()}])
                should_tool_calls.append(["reinitialize_idoc", {"idoc_id": x["idoc_id"]}])

    if x["process"] == "5.0":
        matches = re.findall(r"System status CRTD is active \(ORD ([A-Z0-9]+)", x["error_message"])
        if len(matches) == 1:
            order_number = matches[0]
            should_tool_calls.append(["get_field_by_rpa", {"transaction": "IW33", "rpa_parameters": {"ORDER_NUMBER": matches[0]}, "field": "PERSON_RESPONSIBLE"}])
            should_tool_calls.append(["rfc", {"function": "RFC_READ_TABLE", "rfc_parameters": {"TABLE_NAME": "PUSER002", "PUSER_ID": mockups["RPA_FIELD_VALUE"]["IW33"]["ORDER_NUMBER"][order_number]["PERSON_RESPONSIBLE"]}}])
            should_tool_calls.append(["send_email", {"email": mockups["RFC_READ_TABLE"]["RFC_READ_TABLE"]["PUSER002"]["PUSER_ID"][mockups["RPA_FIELD_VALUE"]["IW33"]["ORDER_NUMBER"][order_number]["PERSON_RESPONSIBLE"]], "content": matches[0]}])
        
        matches = re.findall(r"System status CLSD is active \(ORD ([A-Z0-9]+)", x["error_message"])
        if len(matches) == 1:
            order_number = matches[0]
            should_tool_calls.append(["transaction_SA38", {"program": "RC1_IDOC_SET_STATUS", "program_parameters_dict": {"IDOC_ID": x["idoc_id"], "STATUS": "LKD"}}])
            should_tool_calls.append(["send_email", {"email": "employee3@serviceprovider.com", "content": x["pollex_id"]}])
        
        if "CRTD" not in x["error_message"] and "CLSD" not in x["error_message"]:
            should_tool_calls.append(["send_email", {"email": "Email to error.handler1@company.com", "content": x["idoc_id"]}])

    if x["process"] == "6.0":
        should_tool_calls.append(["transaction_SA38", {"program": "RC1_IDOC_SET_STATUS", "program_parameters_dict": {"IDOC_ID": x["idoc_id"], "STATUS": "LKD"}}])
        should_tool_calls.append(["send_email", {"email": "employee3@serviceprovider.com", "content": x["pollex_id"]}])
    
    if x["process"] == "7.0":
        should_tool_calls = []
        should_tool_calls.append(["reinitialize_idoc", {"idoc_id": x["idoc_id"]}])
    
    if x["process"] == "8.0":
        should_tool_calls.append(["send_email", {"email": "error.handler1@company.com", "content": x["idoc_id"]}])

    if x["process"] == "9.0":
        should_tool_calls = json.loads(x["should_tool_calls"])
    
    return actual_tool_calls, should_tool_calls

In [ ]:
results_dir = "results"
vector_db_dir = "vector_db"
simulation_data = pd.read_csv("error_cases_translated.csv", dtype=str)
simulation_results = []
for i, row in simulation_data.iterrows():
    print(f"Processing row {i}")
    SEEDS = ["25", "114", "281", "654", "759"]
    
    for seed in SEEDS:
        configs = [
            {
                "model": "gpt-4o",
                "retriever_model": "text-embedding-3-small",
                "temperature": 0,
                "seed": 42,
            },
        ]

        for config in configs:
            config["seed"] = seed
            embedder = OpenAIEmbeddings(
            model=config["retriever_model"],
            )
            db = FAISS.load_local(
                os.path.join(vector_db_dir, seed),
                embedder, 
                allow_dangerous_deserialization=True
            )
            RETRIEVER = db.as_retriever()

            del config["retriever_model"]

            results_file = f"simulation_results_rules.csv"
            results_path = os.path.join(results_dir, results_file)
            
            query = row["query"]
            mockups = row["mockups"]
            mockups = json.loads(mockups)
            STORAGE_LOCATIONS_IN_PLANT = mockups["STORAGE_LOCATIONS_IN_PLANT"]
            STORAGE_LOCATION_INFO = mockups["STORAGE_LOCATION_INFO"]
            POLLEX_STORAGE_LOCATIONS = mockups["POLLEX_STORAGE_LOCATIONS"]
            STORAGE_LOCATION_ID = mockups["STORAGE_LOCATION_ID"]
            IDOC_VALUES = mockups["IDOC_VALUES"]
            MATERIALS_IN_STORAGE_LOCATION = mockups["MATERIALS_IN_STORAGE_LOCATION"]
            RFC_READ_TABLE = mockups["RFC_READ_TABLE"]
            RPA_FIELD_VALUE = mockups["RPA_FIELD_VALUE"]

            IDOC_ID = row["idoc_id"]
            POLLEX_ID = row["pollex_id"]
            CURRENT_DATE = mockups["CURRENT_DATE"]

            sys_msg = SystemMessage(content=SYSTEM_PROMPT.format(CURRENT_DATE=CURRENT_DATE))

            react_agent = create_react_agent(config=config, sys_msg=sys_msg, tools=tools)
            st = time.time()
            messages = [HumanMessage(content=query)]
            messages = react_agent.invoke({"messages": messages})
            time_to_complete = time.time() - st

            input_tokens_sum = 0
            cached_tokens_sum = 0
            output_tokens_sum = 0
            price = 0
            messages_text = ""
            rule = ""
            for m in messages["messages"]:
                if type(m) == ToolMessage:
                    if m.name == "retrieve_most_similar_rule":
                        rule = m.content
                if type(m) == AIMessage:
                    token_usage = m.response_metadata.get("token_usage", {})
                    input_tokens = token_usage.get("prompt_tokens", 0)
                    cached_tokens = token_usage.get("prompt_tokens_details", {}).get("cached_tokens", 0)
                    output_tokens = token_usage.get("completion_tokens", 0)
                    input_tokens_sum += input_tokens
                    cached_tokens_sum += cached_tokens
                    output_tokens_sum += output_tokens
                    price += (input_tokens - cached_tokens) * 250 / 1000000 + cached_tokens * 125 / 1000000 + output_tokens * 1000 / 1000000
                messages_text += m.pretty_repr() + "\n"
            
            actual_tool_calls, should_tool_calls =  generate_actual_should(row, messages)

            correct_steps = []
            incorrect_steps = []
            for actual_tool_call in actual_tool_calls:
                if actual_tool_call in should_tool_calls:
                    correct_steps.append(actual_tool_call)
                else:
                    if actual_tool_call[0] == "send_email":
                        should_send_mail_steps = [x for x in should_tool_calls if x[0] == "send_email"]
                        if len(should_send_mail_steps) > 0:
                            should_send_mail_step = should_send_mail_steps[0]
                            if should_send_mail_step[1]["content"] in actual_tool_call[1]["content"] and actual_tool_call[1]["email"] in should_send_mail_step[1]["email"]:
                                correct_steps.append(actual_tool_call)
                            else:
                                incorrect_steps.append(actual_tool_call)
                    else:
                        incorrect_steps.append(actual_tool_call)


            if len(correct_steps) == len(should_tool_calls):
                completed = True
            else:
                completed = False
                print(messages_text)
                print("\n\n#####\n\n")
                print(json.dumps(actual_tool_calls, indent=4))
                print(json.dumps(should_tool_calls, indent=4))
                print("Correct Steps")
                print(json.dumps(correct_steps, indent=4))
                print("Incorrect Steps")
                print(json.dumps(incorrect_steps, indent=4))
                print(price)
            
            failed_attempts = len(incorrect_steps)

            new_row = [row[col] for col in simulation_data.columns]
            new_row.append(messages_text)
            new_row.append(input_tokens_sum)
            new_row.append(cached_tokens_sum)
            new_row.append(output_tokens_sum)
            new_row.append(json.dumps(actual_tool_calls))
            new_row.append(json.dumps(should_tool_calls))
            new_row.append(json.dumps(correct_steps))
            new_row.append(json.dumps(incorrect_steps))
            new_row.append(completed)
            new_row.append(failed_attempts)
            new_row.append(price)
            new_row.append(time_to_complete)
            new_row.append(config["model"])
            new_row.append(rule)
            new_row.append(seed)
            
            simulation_results.append(new_row)
            simulation_results_df = pd.DataFrame(simulation_results, columns=simulation_data.columns.tolist() + ["messages", "input_tokens", "cached_tokens", "output_tokens", "actual_tool_calls", "should_tool_calls", "correct_steps", "incorrect_steps", "completed", "failed_attempts", "price", "time_to_complete", "model", "rule", "seed"])
            simulation_results_df.to_csv(results_path, index=False)
            print(f"Completed row {i} with seed {seed} in {time_to_complete} seconds")